In [2]:
import shapefile
from shapely.geometry import Point, shape
from shapely.geometry.polygon import Polygon

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

import pprint
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_score
from sklearn import metrics

%matplotlib inline

In [13]:
ricel = pd.read_csv("/Users/macbook/Documents/BTP/Notebook/BTP/Satellite/Rice_large.csv")
ricel.head()

,State_Name,ind_district,Crop_Year,Season,Crop,Area,Production,Value
0,Chandigarh,CHANDIGARH,2002,Kharif,Rice,140,700.0,5.0
1,Chandigarh,CHANDIGARH,2003,Kharif,Rice,100,500.0,5.0
2,Chandigarh,CHANDIGARH,2004,Kharif,Rice,80,400.0,5.0
3,Chandigarh,CHANDIGARH,2005,Kharif,Rice,50,250.0,5.0
4,Chandigarh,CHANDIGARH,2006,Kharif,Rice,50,250.0,5.0


In [14]:
a = np.empty((ricel.shape[0],1))*np.NAN
rice = ricel.assign(X1 = a)
rice = rice.assign(X2 = a)
rice.head()

,State_Name,ind_district,Crop_Year,Season,Crop,Area,Production,Value,X1,X2
0,Chandigarh,CHANDIGARH,2002,Kharif,Rice,140,700.0,5.0,NaN,NaN
1,Chandigarh,CHANDIGARH,2003,Kharif,Rice,100,500.0,5.0,NaN,NaN
2,Chandigarh,CHANDIGARH,2004,Kharif,Rice,80,400.0,5.0,NaN,NaN
3,Chandigarh,CHANDIGARH,2005,Kharif,Rice,50,250.0,5.0,NaN,NaN
4,Chandigarh,CHANDIGARH,2006,Kharif,Rice,50,250.0,5.0,NaN,NaN


In [15]:
l = rice.shape[0]
for row in range(0,l):
    if row-1<0 or rice.iloc[row,1] != rice.iloc[row-1,1]:
        continue
    else:
        rice.iloc[row,8] = rice.iloc[row-1,6]
        if row-2<0 or rice.iloc[row,1] != rice.iloc[row-2,1]:
            continue
        else:
            rice.iloc[row,9] = rice.iloc[row-2,6]

In [16]:
rice.head()

,State_Name,ind_district,Crop_Year,Season,Crop,Area,Production,Value,X1,X2
0,Chandigarh,CHANDIGARH,2002,Kharif,Rice,140,700.0,5.0,NaN,NaN
1,Chandigarh,CHANDIGARH,2003,Kharif,Rice,100,500.0,5.0,700.0,NaN
2,Chandigarh,CHANDIGARH,2004,Kharif,Rice,80,400.0,5.0,500.0,700.0
3,Chandigarh,CHANDIGARH,2005,Kharif,Rice,50,250.0,5.0,400.0,500.0
4,Chandigarh,CHANDIGARH,2006,Kharif,Rice,50,250.0,5.0,250.0,400.0


In [17]:
def func(s):
    x = s.strip()
    return x.lower()

In [18]:
rice['ind_district'] = rice['ind_district'].apply(func)
rice['Season'] = rice['Season'].apply(func)
rice.head()

,State_Name,ind_district,Crop_Year,Season,Crop,Area,Production,Value,X1,X2
0,Chandigarh,chandigarh,2002,kharif,Rice,140,700.0,5.0,NaN,NaN
1,Chandigarh,chandigarh,2003,kharif,Rice,100,500.0,5.0,700.0,NaN
2,Chandigarh,chandigarh,2004,kharif,Rice,80,400.0,5.0,500.0,700.0
3,Chandigarh,chandigarh,2005,kharif,Rice,50,250.0,5.0,400.0,500.0
4,Chandigarh,chandigarh,2006,kharif,Rice,50,250.0,5.0,250.0,400.0


In [19]:
ricex = rice[np.isfinite(rice["Production"])]
ricex = ricex[np.isfinite(ricex["X1"])]
ricex = ricex[np.isfinite(ricex["X2"])]
ricex.head()

,State_Name,ind_district,Crop_Year,Season,Crop,Area,Production,Value,X1,X2
2,Chandigarh,chandigarh,2004,kharif,Rice,80,400.0,5.0,500.0,700.0
3,Chandigarh,chandigarh,2005,kharif,Rice,50,250.0,5.0,400.0,500.0
4,Chandigarh,chandigarh,2006,kharif,Rice,50,250.0,5.0,250.0,400.0
5,Chandigarh,chandigarh,2007,kharif,Rice,50,250.0,5.0,250.0,250.0
6,Chandigarh,chandigarh,2008,kharif,Rice,20,100.0,5.0,250.0,250.0


In [20]:
ricex.describe()

,Crop_Year,Area,Production,Value,X1,X2
count,837.000000,837.000000,8.370000e+02,837.000000,8.370000e+02,8.370000e+02
mean,2007.005974,67246.301075,1.708290e+05,2.178032,1.716085e+05,1.689463e+05
std,2.006261,70316.628823,2.300718e+05,0.905854,2.330942e+05,2.319305e+05
min,2004.000000,1.000000,2.000000e+00,0.221533,2.000000e+00,0.000000e+00
25%,2005.000000,9395.000000,1.400000e+04,1.567065,1.400000e+04,1.248500e+04
50%,2007.000000,41868.000000,9.100000e+04,2.069998,8.943300e+04,8.700000e+04
75%,2009.000000,114107.000000,2.610000e+05,2.549000,2.632940e+05,2.570980e+05
max,2010.000000,400000.000000,1.710000e+06,6.315789,1.710000e+06,1.710000e+06


In [21]:
ricex.to_csv("Ricep_large.csv")

In [4]:
point = Point(0.5, 0.5)
polygon = Polygon([(0, 0), (0, 1), (1, 1), (1, 0)])
print(polygon.contains(point))

True


In [5]:
type(polygon)

shapely.geometry.polygon.Polygon

In [6]:
base = "/Users/macbook/Documents/BTP/Satellite/Data/Maps/Districts/Census_2011"

In [7]:
sf = shapefile.Reader(base+"/2011_Dist.shp")

In [28]:
type(sf)
print sf.fields

[('DeletionFlag', 'C', 1, 0), ['DISTRICT', 'C', 28, 0], ['ST_NM', 'C', 24, 0], ['ST_CEN_CD', 'N', 9, 0], ['DT_CEN_CD', 'N', 9, 0], ['censuscode', 'N', 14, 0]]


In [37]:
records = sf.records()
records

[['Adilabad', 'Andhra Pradesh', 28, 1, 532],
 ['Agra', 'Uttar Pradesh', 9, 15, 146],
 ['Ahmadabad', 'Gujarat', 24, 7, 474],
 ['Ahmadnagar', 'Maharashtra', 27, 26, 522],
 ['Aizawl', 'Mizoram', 15, 3, 283],
 ['Ajmer', 'Rajasthan', 8, 21, 119],
 ['Akola', 'Maharashtra', 27, 5, 501],
 ['Alappuzha', 'Kerala', 32, 11, 598],
 ['Aligarh', 'Uttar Pradesh', 9, 12, 143],
 ['Alirajpur', 'Madhya Pradesh', 23, 48, 465],
 ['Allahabad', 'Uttar Pradesh', 9, 44, 175],
 ['Almora', 'Uttarakhand', 5, 9, 64],
 ['Alwar', 'Rajasthan', 8, 6, 104],
 ['Ambala', 'Haryana', 6, 2, 70],
 ['Ambedkar Nagar', 'Uttar Pradesh', 9, 47, 178],
 ['Amravati', 'Maharashtra', 27, 7, 503],
 ['Amreli', 'Gujarat', 24, 13, 480],
 ['Amritsar', 'Punjab', 3, 15, 49],
 ['Anand', 'Gujarat', 24, 15, 482],
 ['Anantapur', 'Andhra Pradesh', 28, 22, 553],
 ['Anantnag', 'Jammu & Kashmir', 1, 14, 14],
 ['Anjaw', 'Arunanchal Pradesh', 12, 16, 260],
 ['Anugul', 'Odisha', 21, 15, 384],
 ['Anuppur', 'Madhya Pradesh', 23, 44, 461],
 ['Araria', 'Bih

In [18]:
for name in dir(shapes[3]):
    if not name.startswith('__'):
        print(name)

bbox
parts
points
shapeType


In [63]:
# Point(Longitude,Latitude)
pt = Point(74.6,19.78)
pt2 = Point(75.7, 29.1)

In [64]:
import fiona
from shapely.geometry import Point, shape

base = "/Users/macbook/Documents/BTP/Satellite/Data/Maps/Districts/Census_2011"
fc = fiona.open(base+"/2011_Dist.shp")

In [66]:
def reverse_geocode(pt):
    for feature in fc:
        if shape(feature['geometry']).contains(pt):
            print feature['properties']['DISTRICT'], feature['properties']['ST_NM']
            return feature['properties']['DISTRICT']
    return "NRI"

### Finally able to do Reverse Geocoding without using any paid API

--------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------
# Lets try selecting rows from a DataFrame with indexes

In [4]:
import numpy as np
import pandas as pd

In [5]:
# importing the dataset we prepared and saved using Baseline 1 Notebook
ricep = pd.read_csv("/Users/macbook/Documents/BTP/Notebook/BTP/ricep.csv")
ricep.head()

,Unnamed: 0,State_Name,ind_district,Crop_Year,Season,Crop,Area,Production,phosphorus,X1,X2,X3,X4
0,15,Andhra Pradesh,anantapur,1999,kharif,Rice,37991.0,105082.0,0.0,96800.0,75400.0,643.720,881.473
1,16,Andhra Pradesh,anantapur,2000,kharif,Rice,39905.0,117680.0,0.0,105082.0,96800.0,767.351,643.720
2,17,Andhra Pradesh,anantapur,2001,kharif,Rice,32878.0,95609.0,0.0,117680.0,105082.0,579.338,767.351
3,18,Andhra Pradesh,anantapur,2002,kharif,Rice,29066.0,66329.0,0.0,95609.0,117680.0,540.070,579.338
4,21,Andhra Pradesh,anantapur,2005,kharif,Rice,25008.0,69972.0,0.0,85051.0,44891.0,819.700,564.500


In [6]:
ricep = ricep.drop(["Unnamed: 0"],axis=1)
ricep["phosphorus"] = ricep["phosphorus"]*10
ricep["value"] = ricep["Production"]/ricep["Area"]

In [7]:
ricep.head()

,State_Name,ind_district,Crop_Year,Season,Crop,Area,Production,phosphorus,X1,X2,X3,X4,value
0,Andhra Pradesh,anantapur,1999,kharif,Rice,37991.0,105082.0,0.0,96800.0,75400.0,643.720,881.473,2.765971
1,Andhra Pradesh,anantapur,2000,kharif,Rice,39905.0,117680.0,0.0,105082.0,96800.0,767.351,643.720,2.949004
2,Andhra Pradesh,anantapur,2001,kharif,Rice,32878.0,95609.0,0.0,117680.0,105082.0,579.338,767.351,2.907993
3,Andhra Pradesh,anantapur,2002,kharif,Rice,29066.0,66329.0,0.0,95609.0,117680.0,540.070,579.338,2.282013
4,Andhra Pradesh,anantapur,2005,kharif,Rice,25008.0,69972.0,0.0,85051.0,44891.0,819.700,564.500,2.797985


In [8]:
ricep.index[ ricep['ind_district'] == 'anantapur'].tolist()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [9]:
ricep.index[ ricep['ind_district'] == 'anantapur' & ricep['Crop_Year'] == '2002' ]

TypeError: cannot compare a dtyped [int64] array with a scalar of type [bool]

In [10]:
ricep.index[ (ricep['ind_district'] == 'anantapur') & (ricep['Crop_Year'] == 2002) ].tolist()

[3]

### So we had to just put the parentheses

In [11]:
a = np.empty((ricep.shape[0],1))*np.NAN
ricex = ricep.assign(test = a)
ricex.head()

,State_Name,ind_district,Crop_Year,Season,Crop,Area,Production,phosphorus,X1,X2,X3,X4,value,test
0,Andhra Pradesh,anantapur,1999,kharif,Rice,37991.0,105082.0,0.0,96800.0,75400.0,643.720,881.473,2.765971,NaN
1,Andhra Pradesh,anantapur,2000,kharif,Rice,39905.0,117680.0,0.0,105082.0,96800.0,767.351,643.720,2.949004,NaN
2,Andhra Pradesh,anantapur,2001,kharif,Rice,32878.0,95609.0,0.0,117680.0,105082.0,579.338,767.351,2.907993,NaN
3,Andhra Pradesh,anantapur,2002,kharif,Rice,29066.0,66329.0,0.0,95609.0,117680.0,540.070,579.338,2.282013,NaN
4,Andhra Pradesh,anantapur,2005,kharif,Rice,25008.0,69972.0,0.0,85051.0,44891.0,819.700,564.500,2.797985,NaN


#### Time taken to execute a cell

In [14]:
%time

v = ricex.iloc[0,13]
v

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.01 µs


nan

In [15]:
d = v + 5

In [16]:
d

nan

In [17]:
if pd.isnull(v):
    v = 3

In [18]:
v+5

8

In [19]:
v

3

In [20]:
df = pd.DataFrame(np.arange(1,7).reshape(2,3),
                  columns = list('abc'), 
                  index=pd.Series([2,5], name='b'))
df

,a,b,c
b,,,
2,1,2,3
5,4,5,6


In [1]:
x = 3

In [2]:
x += 2

In [3]:
x

5

In [4]:
s = "Akshansh"
s[-4]

'a'

In [6]:
# bx = False
# if bx: continue
# else: bx = True